# 2.2 - Ajuste Supervisado

### GridSearching  -  CrossValidation


![grid](images/grid.png)

![cv](images/cv.ppm)

In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np


from sklearn.datasets import make_circles, load_boston

from sklearn.model_selection import train_test_split as tts

from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.ensemble import RandomForestClassifier as RFC

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [11]:
def grid(modelo, param, cv=5):
    
    grid=GridSearchCV(modelo, param, cv=cv,
                      return_train_score=True, n_jobs=-1)
    
    grid.fit(X_train, y_train)
    
    print('Acierto test: {:.2f}'.format(grid.score(X_test, y_test)))
    print('Acierto train: {:.2f}'.format(grid.score(X_train, y_train)))
    print('Mejores parametros: {}'.format(grid.best_params_))
    print('Mejor acierto cv: {:.2f}'.format(grid.best_score_))
    
    return grid.best_estimator_.fit(X_train, y_train)

In [12]:
X_train, X_test, y_train, y_test = tts(load_boston().data, load_boston().target, test_size=.2)

In [13]:
rfr=RFR()

rfr.fit(X_train, y_train)

train_score=rfr.score(X_train, y_train)    # R2
test_score=rfr.score(X_test, y_test)

print (train_score, test_score)

0.9805280727252057 0.8637192726380196


In [14]:
param={'max_leaf_nodes': [5, 10, 20], 'n_estimators': [10, 50, 100, 500]}

In [15]:
grid(rfr, param)

Acierto test: 0.85
Acierto train: 0.94
Mejores parametros: {'max_leaf_nodes': 20, 'n_estimators': 100}
Mejor acierto cv: 0.83


RandomForestRegressor(max_leaf_nodes=20)

In [16]:
modelo=grid(rfr, param)

Acierto test: 0.86
Acierto train: 0.94
Mejores parametros: {'max_leaf_nodes': 20, 'n_estimators': 500}
Mejor acierto cv: 0.83


In [17]:
modelo.predict(X_test)[:10]

array([24.75794885, 16.17629428, 20.53003253, 14.91658992, 18.7403679 ,
       21.53482938, 21.00602678, 12.62095701, 33.07530509, 47.60181539])

### Random GridSearching

### HyperOpt (GridSearching bayesiano)